# Gen AI

This page provides the details on the mlflow gen AI module.

In [1]:
import mlflow
import logging
from mlflow.tracking import MlflowClient
from IPython.display import clear_output

import logging
from langchain_ollama import ChatOllama

logging.basicConfig(level=logging.WARNING)

!rm -f /tmp/llm_mlflow.db
mlflow.set_registry_uri("sqlite:////tmp/llm_mlflow.db")
mlflow.set_tracking_uri("sqlite:////tmp/llm_mlflow.db")

## Trace

Each invocation to the LLM based pipeline is called "trace".

Trace cosits of:

- [**Trace info**](https://mlflow.org/docs/3.3.0/genai/tracing/concepts/trace/#traceinfo-metadata-and-context): general information about the trace primarly used for models ordering and selection.
- [**Trace data**](https://mlflow.org/docs/3.3.0/genai/tracing/concepts/trace/#tracedata-container-of-spans): detailed information about the pipline run. Consists of spans.

---

The following cell runs the experiment that produces the traces we will consider later.

In [2]:
mlflow.langchain.autolog()
chat = ChatOllama(model="llama3.2:1b", temperature=0)
ans = chat.invoke("Hello how are you?")
clear_output()

With `mlflow.search_traces` you can get the traces registered in your mlflows as `pandas.DataFrame`.

In [3]:
traces = mlflow.search_traces()
traces

,trace_id,trace,client_request_id,state,request_time,execution_duration,request,response,trace_metadata,tags,spans,assessments
0,tr-06f1ef26ee7df2eee94b73b1af90c415,"{""info"": {""trace_id"": ""tr-06f1ef26ee7df2eee94b...",None,TraceState.OK,1764150102464,3665,"[[{'content': 'Hello how are you?', 'additiona...","{'generations': [[{'text': ""I'm doing well, th...","{'mlflow.user': 'user', 'mlflow.source.name': ...",{'mlflow.artifactLocation': '/home/user/Docume...,"[{'trace_id': 'BvHvJu598u7pS3Oxr5DEFQ==', 'spa...",[]


The following cell loads a specific trace.

In [4]:
trace_obj = mlflow.get_trace(traces["trace_id"].iloc[0])

The following cell shows loads the `info` of the trace.

In [5]:
trace_obj.info

TraceInfo(trace_id='tr-06f1ef26ee7df2eee94b73b1af90c415', trace_location=TraceLocation(type=<TraceLocationType.MLFLOW_EXPERIMENT: 'MLFLOW_EXPERIMENT'>, mlflow_experiment=MlflowExperimentLocation(experiment_id='0'), inference_table=None), request_time=1764150102464, state=<TraceState.OK: 'OK'>, request_preview='[[{"content": "Hello how are you?", "additional_kwargs": {}, "response_metadata": {}, "type": "human", "name": null, "id": null}]]', response_preview='{"generations": [[{"text": "I\'m doing well, thank you for asking. Is there anything I can help you with or would you like to talk about something in particular?", "generation_info": {"model": "llama3.2:1b", "created_at": "2025-11-26T09:41:46.127363708Z", "done": true, "done_reason": "stop", "total_duration": 3658687704, "load_duration": 1875642829, "prompt_eval_count": 30, "prompt_eval_duration": 394490334, "eval_count": 30, "eval_duration": 1350268885, "model_name": "llama3.2:1b", "model_provider": "ollama"}, "type": "ChatGenerat

This cell displays the `data` from the trace.

In [6]:
trace_obj.data

TraceData(spans=[Span(name='ChatOllama', trace_id='tr-06f1ef26ee7df2eee94b73b1af90c415', span_id='c9eca46a0a240231', parent_id=None)])

## Custom tracing

To build a custom tracing framework, wrap the function with `mlflow.trace` decorator. Any calls of to this function will be tracked in `mlflow`.

---

The following cell creates the traced `some_tracing` function which simply wraps the input in special wrapped text.

In [3]:
mlflow.set_experiment("custom_tracing")

@mlflow.trace
def some_tracing(inp: str) -> str:
    return f"<extra information>{inp}<the data>"

some_tracing("hello")
clear_output()

The following cell shows the kind of span that would be obtained from the corresponding trace.

In [14]:
trace_id = mlflow.search_traces()["trace_id"].iloc[0]
trace = mlflow.get_trace(trace_id=trace_id)

span = trace.data.spans[0]
print(f"{span.inputs} -> {span.outputs}")

{'inp': 'hello'} -> <extra information>hello<the data>
